In [1]:
import re
from pathlib import Path
import glob
import os

def _read_text_any_encoding(path):
    raw = Path(path).read_bytes()

    if raw.startswith(b'\xff\xfe') or raw.startswith(b'\xfe\xff'):
        encoding = 'utf-16'
    elif raw.startswith(b'\xef\xbb\xbf'):
        encoding = 'utf-8-sig'
    else:
        encoding = None
        for enc in ('utf-8', 'cp1252', 'latin-1'):
            try:
                raw.decode(enc)
                encoding = enc
                break
            except UnicodeDecodeError:
                continue
        if encoding is None:
            encoding = 'latin-1'

    return raw.decode(encoding, errors='replace')

def extract_d_rows(filename):
    text = _read_text_any_encoding(filename)
    extracted = []

    for line in text.splitlines():
        line = line.replace('\x00', '')
        line = line.lstrip('\ufeff\u200b\u2060\xa0 \t')  

        if re.match(r'^[Dd]', line):
            values = [v for v in re.split(r'[,\t; ]+', line) if v]
            extracted.append(values)

    return extracted


def process_all_txt_files(folder_path):
    data_dict = {}

    for filepath in glob.glob(os.path.join(folder_path, "*.txt")):
        key = os.path.splitext(os.path.basename(filepath))[0]

        # Extract rows
        rows = extract_d_rows(filepath)

        # Store MPa, lat, long in list
        extracted = []
        for r in rows:
            try:
                extracted.append((float(r[-4]), float(r[-3]), float(r[-2])))
            except (ValueError, IndexError):
                continue

        data_dict[key] = extracted

    return data_dict

folder = r"C:\Users\telukkari\Documents\Data\Lieksa_kantavuusmittaukset"
all_data = process_all_txt_files(folder)

for k, v in all_data.items():
    print(f"{k}: {len(v)} rows")
    print(v[:3])

Hanhivaarantie_Hattuvaara: 110 rows
[(198.0, 6315.86237, 3026.964), (155.0, 6315.8835, 3026.92889), (51.0, 6315.90743, 3026.94885)]
Inkeriläntie_Isonvaara: 103 rows
[(159.0, 6317.72417, 3017.08722), (151.0, 6317.74614, 3017.12054), (66.0, 6317.76505, 3017.16122)]
Jaakonvaarantiehaara_Emo: 126 rows
[(177.0, 6310.75642, 3014.83016), (195.0, 6310.73377, 3014.79944), (44.0, 6310.71073, 3014.76907)]
Larinvaarantie_Oinola: 53 rows
[(145.0, 6311.13509, 3031.74244), (29.0, 6311.16063, 3031.75933), (83.0, 6311.18317, 3031.79604)]
Lipinvaarantie_Luhovaara: 242 rows
[(189.0, 6310.99479, 3028.38937), (255.0, 6310.97872, 3028.45059), (99.0, 6310.96833, 3028.49145)]
Ranta-ahontie haara_julo: 14 rows
[(103.0, 6311.37578, 3029.10358), (25.0, 6311.40106, 3029.12302), (49.0, 6311.42657, 3029.14272)]
Suontauksentie_Kontiovaara: 83 rows
[(50.0, 6310.51619, 3033.1717), (52.0, 6310.4996, 3033.12458), (37.0, 6310.48383, 3033.07668)]


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from rasterio.warp import transform
from rasterio.transform import array_bounds
from shapely.geometry import Point
from rasterio.plot import show
import geopandas as gpd
from pyproj import Transformer
import laspy

def nmea_to_decimal(coord, hemisphere):
    coord = float(coord)
    if hemisphere in ['N','S']:
        degrees = int(coord // 100)
        minutes = coord - degrees * 100
    else:  
        degrees = int(coord // 100)
        minutes = coord - degrees * 100
    
    decimal = degrees + minutes / 60.0
    
    if hemisphere in ['S','W']:
        decimal = -decimal
    return decimal

for key, data in all_data.items():
    print(f"Processing {key} with {len(data)} points")
    mpa = np.array([t[0] for t in data], dtype=float)
    lons = np.array([nmea_to_decimal(t[1], 'E')  for t in data], dtype=float)
    lats = np.array([nmea_to_decimal(t[2], 'N') for t in data], dtype=float)
    to_tm35 = Transformer.from_crs("EPSG:4326", "EPSG:3067", always_xy=True)
    xs, ys = to_tm35.transform(lats, lons)

    print(xs[0], ys[0])

    # save one sample measurements for visualization

    header = laspy.LasHeader(version="1.4", point_format=6)
    header.scales = [0.0001, 0.0001, 0.0001]

    
    zeros = np.zeros_like(xs)

    stacked = np.column_stack((xs, ys, zeros))

    header.offsets = np.mean(stacked, axis=0)    # shape (3,)

    las = laspy.LasData(header)

    las.x = np.array(xs)
    las.y = np.array(ys)

    las.add_extra_dim(laspy.ExtraBytesParams(name="bearing_capacity",  type=np.int16))
    las.bearing_capacity = np.array(mpa).astype(np.int16) 

    las.write(f'measurements_{key}.las')


Processing Hanhivaarantie_Hattuvaara with 110 points
673076.7284292739 7019700.315328252
Processing Inkeriläntie_Isonvaara with 103 points
664645.5826654527 7022720.629820288
Processing Jaakonvaarantiehaara_Emo with 126 points
663416.0960348095 7009697.722593522
Processing Larinvaarantie_Oinola with 53 points
677552.5905328043 7011149.049043946
Processing Lipinvaarantie_Luhovaara with 242 points
674757.0955352939 7010735.383325999
Processing Ranta-ahontie haara_julo with 14 points
675317.1880220661 7011474.579113952
Processing Suontauksentie_Kontiovaara with 83 points
678813.7701847918 7010067.2006459255
